## EXAMEN EDA

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### DATASET

In [16]:
df_forestCovtype = pd.read_csv("covtype.data")

In [17]:
# Poner el nombre de las columnas en el dataset

titulo_columnas = ["Elevation", "Aspect", "Slope", "Horizontal_Distance_To_Hydrology",
            "Vertical_Distance_To_Hydrology", "Horizontal_Distance_To_Roadways",
            "Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm",
            "Horizontal_Distance_To_Fire_Points"]
wilderness_columnas = ["Wilderness_Area_"+str(i) for i in range(1,5)]
soil_columnas = ["Soil_Type_"+str(i) for i in range(1,41)]

titulo_columnas += wilderness_columnas + soil_columnas + ["Cover_Type"]
df_forestCovtype.columns = titulo_columnas

df_forestCovtype

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,Soil_Type_40,Cover_Type
0,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
1,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
2,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
3,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5
4,2579,132,6,300,-15,67,230,237,140,6031,...,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581006,2396,153,20,85,17,108,240,237,118,837,...,0,0,0,0,0,0,0,0,0,3
581007,2391,152,19,67,12,95,240,237,119,845,...,0,0,0,0,0,0,0,0,0,3
581008,2386,159,17,60,7,90,236,241,130,854,...,0,0,0,0,0,0,0,0,0,3
581009,2384,170,15,60,5,90,230,245,143,864,...,0,0,0,0,0,0,0,0,0,3


In [18]:
# Unir las  columnas de wilderness y soil en una sola columna, sumando los valores para ver de cuantos tipos hay

df_forestCovtype["Wilderness_Area"] = df_forestCovtype[wilderness_columnas].max(axis = 1)
df_forestCovtype.drop(wilderness_columnas, axis = 1, inplace = True)

df_forestCovtype["Soil_Type"] = df_forestCovtype[soil_columnas].max(axis = 1)
df_forestCovtype.drop(soil_columnas, axis = 1, inplace = True )

df_forestCovtype = df_forestCovtype.reset_index(drop = True)
df_forestCovtype

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Cover_Type,Wilderness_Area,Soil_Type
0,2590,56,2,212,-6,390,220,235,151,6225,5,1,1
1,2804,139,9,268,65,3180,234,238,135,6121,2,1,1
2,2785,155,18,242,118,3090,238,238,122,6211,2,1,1
3,2595,45,2,153,-1,391,220,234,150,6172,5,1,1
4,2579,132,6,300,-15,67,230,237,140,6031,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
581006,2396,153,20,85,17,108,240,237,118,837,3,1,1
581007,2391,152,19,67,12,95,240,237,119,845,3,1,1
581008,2386,159,17,60,7,90,236,241,130,854,3,1,1
581009,2384,170,15,60,5,90,230,245,143,864,3,1,1


In [19]:
df_forestCovtype.isna().sum()

Elevation                             0
Aspect                                0
Slope                                 0
Horizontal_Distance_To_Hydrology      0
Vertical_Distance_To_Hydrology        0
Horizontal_Distance_To_Roadways       0
Hillshade_9am                         0
Hillshade_Noon                        0
Hillshade_3pm                         0
Horizontal_Distance_To_Fire_Points    0
Cover_Type                            0
Wilderness_Area                       0
Soil_Type                             0
dtype: int64

In [20]:
df_forestCovtype.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581011 entries, 0 to 581010
Data columns (total 13 columns):
 #   Column                              Non-Null Count   Dtype
---  ------                              --------------   -----
 0   Elevation                           581011 non-null  int64
 1   Aspect                              581011 non-null  int64
 2   Slope                               581011 non-null  int64
 3   Horizontal_Distance_To_Hydrology    581011 non-null  int64
 4   Vertical_Distance_To_Hydrology      581011 non-null  int64
 5   Horizontal_Distance_To_Roadways     581011 non-null  int64
 6   Hillshade_9am                       581011 non-null  int64
 7   Hillshade_Noon                      581011 non-null  int64
 8   Hillshade_3pm                       581011 non-null  int64
 9   Horizontal_Distance_To_Fire_Points  581011 non-null  int64
 10  Cover_Type                          581011 non-null  int64
 11  Wilderness_Area                     581011 non-null 

In [21]:
df_forestCovtype.drop(df_forestCovtype.dropna().index)

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Cover_Type,Wilderness_Area,Soil_Type


### EJERCICIO 1

Para conseguir un dataset con una dimensión reducidad, aplica la técnica de Selección de variables basada en árbol de decisión mediante las importancias de cada variable (Decision Trees Importances):

- Filtra el tablón para quedarnos solamente con las variables que aglutinan hasta el 95% de la información que se requiere para estimar la variable objetivo.
- random_state=100